In [ ]:
from nd2reader import ND2Reader
import numpy as np
import nrrd
import os

def readtxt(filename):
    a = open(filename)
    content = a.readlines()
    a.close()
    return content

def nd2ToVol(filename, channel_name='405', ratio=1):
    vol = ND2Reader(filename)
    channel_names = vol.metadata['channels']
    channel_id = [x for x in range(len(channel_names)) if channel_name in channel_names[x]]
    assert len(channel_id) == 1
    channel_id = channel_id[0]
    
    out = np.zeros([len(vol), vol[0].shape[0] // ratio, vol[0].shape[1] // ratio], np.uint16)
    for z in range(len(vol)):
        out[z] = vol.get_frame_2D(c=channel_id, t=0, z=z, x=0, y=0, v=0)[::ratio, ::ratio]
    return out

De = './elastix-4.9.0-linux/'

In [ ]:
nd2s = ['/src/src/20211130_larva1_barcode3.nd2', '/src/src/20211130_larva1_barcode5.nd2']
nrrds = ['/src/src/20211130_larva1_barcode3.nrrd', '/src/src/20211130_larva1_barcode5.nrrd']
output = '/src/src/20211130_larva1_barcode5/aligned_/'

print('channels:', ND2Reader(nd2s[0]).metadata['channels'])

for nid in range(2):
    if not os.path.exists(nrrds[nid]):
        vol = nd2ToVol(nd2s[nid], channel_name='405', ratio=1)
        nrrd.write(nrrds[nid], vol, header={'spacings': [4,1.625,1.625]})

cmd = '! LD_LIBRARY_PATH=$LD_LIBRARY_PATH:%s/lib; %s/bin/elastix -f %s -m %s -out %s -p Parameters.txt' \
          % (De, De, nrrds[0], nrrds[1], output)
os.system('mkdir %s' %output)
print('copy the command below for execution')
print('---------------')
print(cmd)
print('---------------')

In [ ]:
! LD_LIBRARY_PATH=$LD_LIBRARY_PATH:./elastix-4.9.0-linux/lib; ./elastix-4.9.0-linux/bin/elastix -f 20211130_larva1_barcode3.nrrd -m 20211130_larva1_barcode5.nrrd -out /content/drive/MyDrive/2021-11-29_WORMseq6/WORMSeq6_S3_barcode5/20211130_larva/20211130_larva1_barcode5_aligned/ -p Parameters.txt

In [ ]:
!echo $LD_LIBRARY_PATH

In [ ]:
ff = ND2Reader(nd2s[0])
ff.metadata

In [ ]:
ff.metadata.keys()

In [ ]:
ff.metadata['pixel_microns'], ff.metadata['total_images_per_channel']
ff.metadata['z_coordinates']
ff.metadata['pixel_microns'],ff.metadata['z_levels']

In [ ]:
ff.sizes
dir(ff)